# pip install

In [ ]:
!pip install gdown
!pip install scikit-learn
!pip install joblib

# Import Libraries

In [1]:
from utilities.data_processor import DataPreProcessor, DataLoader
from utilities.XGBoost_model import train_and_evaluate_xgboost
from utilities.XGBoost_Predictor import XGBoostPredictor
import os
import json

ModuleNotFoundError: No module named 'langchain_openai'

# Load Raw Data

In [ ]:
dpp = DataPreProcessor()

# Load the data
dpp.load_data(download=True)

# show the row data info
dpp.show_data_info()

# preprocess the data
dpp.preprocess_data(fill_blank=True, fill_mode='mode')
 
# show the preprocessed data info
dpp.show_data_info()

# plt the remaining features
dpp.show_feature_plt()

# DataLoader

In [ ]:
# 初始化DataLoader
dl = DataLoader(dpp.get_data(), mode='catboost')

# 特征编码
dl.encoder()

# 划分数据集
dl.split_data()

# 筛选特征
dl.data_filter()

x_train_filtered, x_test_filtered, y_train, y_test = dl.get_filtered_data()

# Train and Evaluate XGBoost

In [ ]:
# 训练并评估xgboost
bst, accuracy = train_and_evaluate_xgboost(x_train_filtered, x_test_filtered, y_train, y_test)

# 保存模型和特征中位数
model_path = "model_folder/xgboost_model.json"
stats_path = "model_folder/feature_medians.json"
os.makedirs("model_folder", exist_ok=True)
bst.save_model(model_path)

feature_medians = x_train_filtered.median().to_dict()
with open(stats_path, "w") as f:
    json.dump(feature_medians, f)

print("Model and feature medians saved.")

# Predict

In [ ]:
# 导入预测类
predictor = XGBoostPredictor()

# 加载模型和特征统计信息
predictor.load_model_and_stats("model_folder/xgboost_model.json", "model_folder/feature_medians.json")

# 用户输入数据，字典形式
input_data = {
    "out_prncp_inv": 100,
}

# 预测
try:
    prediction = predictor.predict(input_data)
    print(f"Predicted class: {prediction}")
except ValueError as e:
    print(f"Error: {e}")